## From BNC to Ngram - clarification 
There seems to be a general sense of confusion regarding this homeword assignment. I would try to provide a more comprehensive instruction with some hints to handle some of the problems encountered during class.

## Objective
The end goal of this assignment is to generate the rank and rank ratio between BNC and clang8 for all the bigrams with the format "adj. accident"


#### BNC Data:  
https://drive.google.com/file/d/1mKX1DLHDIqKph4e4k1MnYOV3iWtvT7-E/view?usp=sharing

#### Cleaned Lang 8 Data:
https://drive.google.com/file/d/11wxKJr-VpmrHZ-MR41i4E097uY-T9okH/view?usp=sharing


### A. Processing BNC Data
### 1.1 Extract lines containing id, title, classcode, keywords, sentences from each BNC parts (ABCDEFGHJK)

using grep / egrep to match regular expression and extract relavent data  

Reference
https://www.twblogs.net/a/5d26d705bd9eee1e5c84509d

In [1]:
# extract data from raw data
! egrep -o -h \
'(<idno type="bnc">.*?</idno>|<title>.*?</title>|<classCode.*?</classCode>|<keywords>.*?</keywords>|<s n=".*?">|<w c5=".*?" hw=".*?" pos=".*?">.*?</w>|<c c5=".*?">.*?</c>|</s>|<p>|</p>)' \
Texts/*/*/A*.xml | sed 's/></>\n</g' > BNC.A.txt

egrep -o -h  Texts/*/*/A*.xml > BNC.A.txt  237.80s user 1.29s system 99% cpu 4:00.15 total


**NOTE:** From the lab, it seems most systems output  

\<s n="1"\>\<w c5="NN1" hw="factsheet" pos="SUBST"\>FACTSHEET \</w\>\<w c5="DTQ" hw="what" pos="PRON"\>WHAT \</w\>\<w c5="VBZ" hw="be" pos="VERB"\>IS \</w\>\<w c5="NN1" hw="aids" pos="SUBST"\>AIDS\</w\>...\</s\>  
  
as one line. For line_to_token() to work, please sepereate each word (\<w\> ~ \</w\>) into one line.

In [1]:
! head BNC.A.txt

<title> [ACET factsheets &amp; newsletters]. Sample containing about 6688 words of miscellanea (domain: social science) </title>
<idno type="bnc">A00</idno>
<title> [ACET factsheets &amp; newsletters]. </title>
<classCode scheme="DLEE">W nonAc: medicine</classCode>
<keywords><term> Health </term><term> Sex </term></keywords>
<s n="1">
<w c5="NN1" hw="factsheet" pos="SUBST">FACTSHEET </w>
<w c5="DTQ" hw="what" pos="PRON">WHAT </w>
<w c5="VBZ" hw="be" pos="VERB">IS </w>
<w c5="NN1" hw="aids" pos="SUBST">AIDS</w>


 ### 2. Convert sentences to bigram (for all sections A to K, no I)
 After you extract all the BNC data (BNC.A.txt, BNC.B.txt, BNC.C.txt ...), you need to process xml into tokens and bigrams.
 
 ### 2.1 Convert line to word tokens

In [7]:
import re
from pprint import pprint
BNC_bigram_total_number = 0 
    
def line_to_token(line):
    BNC_bigram_total_number = 0
    if line.startswith('<s'):
        return ('<s> ', '<s>', '<s>') 
    elif line.startswith('</s'):
        return ('</s>', '</s>', '</s>') 
    elif line.startswith('<w'):
        # <w c5="VVN" hw="discount" pos="VERB">discounted </w>
        match = re.findall('<w c5="(.*?)" hw="(.*?)" pos=".*?">(.*?)</w>', line)
        return (match[0][2].strip(), match[0][0].upper(), match[0][1]) #  , tag, word
    elif line.startswith('<c'):
        match = re.findall('<c c5="PUN">(.*?)</c>', line)
        if not match:
            return '??? line'
        return (match[0], match[0], match[0])

def tokens_to_bigram(tokens):
    result = []
    for i in range(len(tokens)-1):
        if i == 1:
            word2tag2lemma2 = [tokens[i][j].lower()+' '+tokens[i+1][j] for j in range(3)]
        else:
            word2tag2lemma2 = [tokens[i][j]+' '+tokens[i+1][j] for j in range(3)]
        if word2tag2lemma2[0][0].isalpha() or word2tag2lemma2[0][0] == '<': 
            result = result + [ '\t'.join(word2tag2lemma2) ]
    return result

### 2.2 Convert token stream to bigram stream

In [8]:
def word_to_bigram(wordfile, bigramfile):
    BNC_bigram_total_number = 0
    def Batch_to_ngram(batch, fileout):        
        with open(wordfile.format(batch),'r',encoding="utf-8") as filein:
            lines = filein.readlines()
            count = 1
            last = 0
            for i, line in enumerate(lines):
                if line.startswith('<s'):
                    sent_start = i
                    count = count + 1
                elif line.startswith('</s'):
                    sentence = [line.strip() for line in lines[sent_start:i+1]]
                    tokens = [line_to_token(line) for line in sentence ]
                    bigram = tokens_to_bigram(tokens)
                    count = count + 1
                    last = i
                    print('\n'.join(bigram), file=fileout)
            BNC_bigram_total_number = last-count
            return BNC_bigram_total_number
    
    with open(bigramfile, 'w',encoding="utf-8") as fileout:
        for batch in 'A':
            BNC_bigram_total_number = Batch_to_ngram(batch, fileout)
    return BNC_bigram_total_number
                
BNC_bigram_total_number = word_to_bigram('BNC.{0}.txt', 'BNC.2w.txt')
print(BNC_bigram_total_number)

13854202


### 3 Sort and count bigram (word1 word2 \<tab\> count) 

In [103]:
#1 BNC.2w.txt ==> BNC.2w.c.txt
# We sort the bigrams and count identical bigrams
# NOTE: BNC.2w.c.txt should be considerably smaller than BNC.2w.txt
# NOTE: Since we only care about "adj. accident", you can filter BNC.2w.c.txt into a much
#       smaller file by extracting bigrams that fit "adj. accident"

! time sort BNC.2w.txt | uniq -c | \
awk '{ gsub(/^[ ]*/, ""); print }' | awk '{print substr($0, index($0, " ")+1) "\t" $1}' > BNC.2w.c.txt

sort: No such file or directory
sort BNC.B.2w.txt  0.00s user 0.00s system 54% cpu 0.005 total
uniq -c  0.00s user 0.00s system 49% cpu 0.004 total
awk '{ gsub(/^[ ]*/, ""); print }'  0.00s user 0.00s system 49% cpu 0.004 total
awk '{print substr($0, index($0, " ")+1) "\t" $1}'  0.00s user 0.00s system 52% cpu 0.004 total
egrep -v '\t1$' > BNC.B.2w.c2+.txt  0.00s user 0.00s system 56% cpu 0.003 total


In [134]:
# Example data format for BNC.2w.c.txt
# bigram       pos      lemmas          count

! egrep '^(big|serious|fatal) accident\t' BNC.2w.c.txt
! egrep 'accident\s.*AJ' BNC.2w.c.txt > adj_accident.txt

big accident	AJ0 NN1	big accident	3
fatal accident	AJ0 NN1	fatal accident	82
fatal accident	aj0 NN1	fatal accident	3
serious accident	AJ0 NN1	serious accident	61


### B. Processing clang8 Data
During the lab, we found out that the lang8 dataset we provided last week did not contain enough "adj. accident" bigrams for this assignment.  
Please process the clang8 data provided above and extract bigrams.  
**NOTE:** We are only intereted in the rank / rank ratio for  "adj. accident" bigrams. Please extract bigrams of "adj. accident" from both BNC and clang8 first, then calculate the rank and rank ratio. 

Target output format: 
https://drive.google.com/file/d/1xM46aaDIeu4Z0FkikGOcmDoq7u2O47tY/view?usp=sharing

Demo time sign up for Lab 3:
https://docs.google.com/spreadsheets/d/1OKbXhcv6E3FEQDPnbHEHEeHvpxv01jxugMP7WwnKqKw/edit?usp=sharing

For Demo, please print out all the "adj. accident" bigrams in descending rank ratio order.



In [9]:
import os
import re
import string
import copy
from pprint import pprint

import itertools
from tabulate import tabulate

In [10]:

def tokenize(text):
    """
    Input:
    "This is an example.'

    Sample output: 
    ['this', 'is', 'an', 'example', '.']
    """  
    #### [ TODO ] transform text to lower case
    lowerText = text.lower()
    pureText = re.sub(r'[.,"\'-?:!;]', '', re.sub(r'[0-9]+', '', lowerText))
    #### [ TODO ] seperate the words by white space
    splitText = pureText.split()
    return splitText
    
from collections import Counter

def calculate_frequency(tokens):
    """
    Input:
    ['this', 'is', 'an', 'example', ...]

    Sample output: 
    {
        'the': 79809, 
        'project': 288,
        ...
    }
    """
    #### [ TODO ] 
    topToken = Counter(tokens).most_common()
    di = dict(topToken)
    return di
   
def ranking(tokens):
    rank = 0
    previous_number = 0
    for element in tokens:
        if(tokens[element] != previous_number):
            rank += 1
        previous_number = tokens[element]
        tokens[element] = rank
    return tokens

def get_ngram(tokens,n):
    ngram_result = []
    ngram_list = [tokens[i:i+n] for i in range(0,len(tokens)-1)]
    for element in ngram_list:
        ngram_result.append(' '.join(element[0 : n]))
    return ngram_result

    """
    Input:
    ['this', 'is', 'an', 'example', ...]

    Sample output: 
    ['this is', 'is an', 'an example', ...]
    """
    #### [TODO] 

In [11]:
### BNC Processing
f = open('adj_accident.txt', 'r',encoding="utf-8")
line = f.readline()
BNC_bigram_counter = {}
while line:
    BNC_readline = line.split()
    BNC_dic_key = BNC_readline[4] +' '+BNC_readline[5]
    BNC_dic_value = int(BNC_readline[6])
    BNC_bigram_counter[BNC_dic_key]=BNC_dic_value
    line = f.readline()

list_BNC_bigram_counter = sorted(BNC_bigram_counter.items(), key=lambda kv: kv[1],reverse=True)
BNC_bigram_counter = dict(list_BNC_bigram_counter)
maintain_BNC_bigram_counter = copy.deepcopy(BNC_bigram_counter)
BNC_bigram_Rank = ranking(BNC_bigram_counter)

f.close()


In [13]:
f = open('clang8.txt', 'r',encoding="utf-8")
fileRead = f.read()
#### [ TODO ] generate lang8 unigrams and calculate document frequency of unigram in lang8
clang_unigram = tokenize(fileRead)
f.close()

clang_bigram = get_ngram(clang_unigram,2)
clang_bigram_total_number = len(clang_bigram)
clang_bigram_counter = calculate_frequency(clang_bigram)
clang_bigram_accident_counter = {}

for element in BNC_bigram_counter:
    if(element in clang_bigram_counter):
        clang_bigram_accident_counter[element]=clang_bigram_counter[element]
list_clang_bigram_counter = sorted(clang_bigram_accident_counter.items(), key=lambda kv: kv[1],reverse=True)
clang_bigram_accident_counter = dict(list_clang_bigram_counter)
maintain_clang_bigram_accident_counter = copy.deepcopy(clang_bigram_accident_counter)
clang_bigram_Rank = ranking(clang_bigram_accident_counter)

In [14]:
print(maintain_BNC_bigram_counter)
print('-----------------------')
print(BNC_bigram_Rank)
print('-----------------------')
print(maintain_clang_bigram_accident_counter)
print('-----------------------')
print(clang_bigram_Rank)

{'serious accident': 20, 'fatal accident': 8, 'nuclear accident': 7, 'personal accident': 7, 'historical accident': 5, 'disabling accident': 3, 'industrial accident': 3, 'lucky accident': 3, 'mere accident': 3, 'catastrophic accident': 2, 'dreadful accident': 2, 'major accident': 2, 'minor accident': 2, 'sad accident': 2, 'similar accident': 2, 'terrible accident': 2, 'tragic accident': 2, 'british accident': 1, 'accident black': 1, 'accident bad': 1, 'appalling accident': 1, 'awful accident': 1, 'bad accident': 1, 'big accident': 1, 'cerebrovascular accident': 1, 'collusive accident': 1, 'common accident': 1, 'complete accident': 1, 'conceivable accident': 1, 'diving accident': 1, 'entertaining accident': 1, 'exceptional accident': 1, 'flying accident': 1, 'follow accident': 1, 'freak accident': 1, 'free accident': 1, 'full accident': 1, 'far accident': 1, 'heavy accident': 1, 'hit-and-run accident': 1, 'huge accident': 1, 'immediate accident': 1, 'likely accident': 1, 'little acciden

In [17]:
#### [ TODO ] Calculate Rank Ratio
BNC_rank_ratio = {}
for element in BNC_bigram_Rank:
    if(element in clang_bigram_Rank):
        BNC_rank_ratio[element] = (BNC_bigram_Rank[element]/clang_bigram_Rank[element])

list_BNC_rank_ratio = sorted(BNC_rank_ratio.items(), key=lambda x: x[1])
for i,word_rank_ratio in enumerate(list_BNC_rank_ratio):
    BNC_usage_ratio = (maintain_BNC_bigram_counter[word_rank_ratio[0]] / BNC_bigram_total_number) * 1000000 
    BNC_usage_ratio = "(%.2f)" % BNC_usage_ratio
    clang_usage_ratio = (maintain_clang_bigram_accident_counter[word_rank_ratio[0]] / clang_bigram_total_number) * 1000000
    clang_usage_ratio = "(%.2f)" % clang_usage_ratio
    BNC_lang_rank = (str(BNC_bigram_Rank[word_rank_ratio[0]])+BNC_usage_ratio,str(clang_bigram_Rank[word_rank_ratio[0]])+clang_usage_ratio)
    list_BNC_rank_ratio[i] = list_BNC_rank_ratio[i] + BNC_lang_rank

headers = ["Phrases", "Overuse rank/rank","BNC rank(#/m)","LANG-8 rank(#/m)"]
print(tabulate( list_BNC_rank_ratio, headers = headers))

Phrases                     Overuse rank/rank  BNC rank(#/m)    LANG-8 rank(#/m)
------------------------  -------------------  ---------------  ------------------
serious accident                     0.2       1(1.44)          5(0.54)
fatal accident                       0.285714  2(0.58)          7(0.25)
historical accident                  0.363636  4(0.36)          11(0.04)
mere accident                        0.454545  5(0.22)          11(0.04)
industrial accident                  0.5       5(0.22)          10(0.08)
catastrophic accident                0.545455  6(0.14)          11(0.04)
major accident                       0.545455  6(0.14)          11(0.04)
lucky accident                       0.555556  5(0.22)          9(0.12)
minor accident                       0.6       6(0.14)          10(0.08)
sad accident                         0.6       6(0.14)          10(0.08)
similar accident                     0.6       6(0.14)          10(0.08)
accident bad                        